# Segementing and Clustering Neighbourhoods in Toronto

## Adding Geospatial Data

import required libraries


In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Request page and find required table

In [7]:
# Used an older version of the wiki page which has an easier to read and process table format
URL = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788"

# generate the page request
page = requests.get(URL)

# parse the html
soup = BeautifulSoup(page.content, 'html.parser')

# Using BeautifulSoup find the required table
table=soup.find('table')


Process table and convert to df

In [8]:
# set up list of rows to convert to df
table_contents=[]

#set iterator to be able to identify <td>'s
i=1

#create empty cell struccture
cell = {}

#for each td in table
for row in table.findAll('td'):
    
    # if in the first column it will be the postal code
    if i % 3 == 1:
        cell['Postal Code'] = row.text.replace('\n','')

    # else if it's in the 2nd column it's the borough
    elif i % 3 == 2:
        cell['Borough'] = row.text.replace('\n','')

    # else if it's in the 3rd column it's the neighbourhood 
    elif i % 3 == 0:
        cell['Neighborhood'] = row.text.replace('\n','')

        #once every 3rd td is reached append the created cell to the list of cells, and generate a new cell
        table_contents.append(cell)
        cell = {}

    i += 1 

# create the df
df=pd.DataFrame(table_contents)

#remove unassigned postcodes from the table
df = df[df.Borough != 'Not assigned']

Shape of the dataframe

In [12]:
gsc = pd.read_csv("Geospatial_Coordinates.csv")

df = pd.merge(df, gsc, on="Postal Code")

df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
